In [2]:
import pandas as pd
import os
from blastScripts import make_queries as mq
import shutil

### Directions:  
1. Need to have directory with proteomes with '.faa' extension.  
    - can also move typical ncbi_datasets file structure and use the function below to pull out the proteomes to their own directory
2. make a text file with all accessions in it
    -this is only necessary if you don't want to make blastp databases from all of the proteomes
3. 


### If starting with ncbi datasets directory structure, move proteomes to their own directory

In [45]:
def move_ncbi_datasets_proteomes(data_folder):
    proteomes_dir = './Proteomes'
    if not os.path.exists(proteomes_dir):
        os.makedirs(proteomes_dir)
    assembly_dirs = [f for f in os.listdir(p_dir) if f.startswith('GC')]
    for assemb_dir in assembly_dirs:
        fp = os.path.join(p_dir,assemb_dir)
        files = os.listdir(fp)
        for file in files:
            if file.endswith('.faa'):
                proteome_fp = os.path.join(fp, file)
                name = assemb_dir + '.faa'
                new_fp = os.path.join(proteomes_dir,name)
                shutil.move(proteome_fp, new_fp )

data_folder =  './ncbi_dataset/ncbi_dataset/data'
#move_ncbi_datasets_proteomes(data_folder)

### accessions of proteomes used to build blastp databases

In [44]:
deduped_accessions_w_palmer_strains = []
with open('./deduped_accessions_w_palmer_strains.txt', 'r') as f:
    for line in f:
        deduped_accessions_w_palmer_strains.append(line.strip())

### Functions

In [12]:
def make_blastp_databases(proteome_dir, out_dir, proteomes_to_include=None):
    """Make a blastp database for each proteome within a given directory"""
    
    if not proteomes_to_include:
        proteomes_to_include = [acc.replace('.faa', '') for acc in os.listdir(proteome_dir)]
    for acc in proteomes_to_include:
        proteome_path = f'{proteome_dir}/{acc}.faa'
        db_name = acc+'_db'
        out = os.path.join(out_dir, acc, db_name)
        !makeblastdb -in $proteome_path -dbtype prot -out $out
        
        
def multi_database_query(database_dir, query_filepath, out_dir_path=None):
    """Query each blastp database within the database_dir and make a hits text file in the out_dir"""
        
    if not out_dir_path:
        out_dir_path = './forward_blast_results'
    if not os.path.exists(out_dir_path):
        os.makedirs(out_dir_path)
    for acc in [acc for acc in os.listdir(database_dir) if acc != '.DS_Store']:
        base_filepath = os.path.join(database_dir, f'{acc}/{acc}_db') #filepath of blastp database to query
        out_filepath = os.path.join(out_dir_path, f'{acc}_blastpOut.txt')
        print(out_filepath)
        !blastp -query $query_filepath -db $base_filepath -num_descriptions 5 -num_alignments 5 -out $out_filepath
        
def recip_blast(rblast_queries_dir, rblast_db_path, outDir):
    """ """
    os.makedirs(outDir)
    query_files = [file for file in os.listdir(rblast_queries_dir) if file.endswith('.faa')]
    for query in query_files:
        outfile_name = query.replace('.faa', '_blastpOut.txt')
        out_path = os.path.join(outDir, f'{outfile_name}')
        query_path  = os.path.join(rblast_queries_dir, query)
        print(query_path,rblast_db_path, out_path, )
        !blastp -query $query_path -db $rblast_db_path -num_descriptions 5 -num_alignments 5 -out $out_path

# ENTER THESE PATHS:  
1. database_dir (str): Enter the desired name of the directory that will house all reciprocal blast results.   
2. proteome_dir (str): path to the directory containin all of the proteomes used to build the blastp databases and to generate reverse blast queries.   
3. reference_proteome_path (str): path to the proteome that the original forward query sequence was derived.   


In [13]:
database_dir = './deduped_baumannii_blast_db'
proteome_dir = './Proteomes'
reference_proteome_path = os.path.join(proteome_dir, 'GCF_001077675.1.faa')

### Make individual genome blast databases from deduped baumannii genomes + Palmer lab baumannii genomes

In [14]:
#make_blastp_databases(proteome_dir, out_dir=database_dir, proteomes_to_include=deduped_accessions_w_palmer_strains)

In [15]:
all_proteomes = [p.replace('.faa', '') for p in os.listdir('./Proteomes')]
len(deduped_accessions_w_palmer_strains)
for a in deduped_accessions_w_palmer_strains:
    p = a.strip() + '.faa'
    if a not in all_proteomes:
        print(a)
        

In [16]:
def reciprocal_blastp(database_dir, query_file, out_dirname, proteome_dir, reference_proteome_path, accessions_to_include):
    """
    
    parameters:
    database_dir (str): Path to the parent directory holding all of the individual database folders.
    out_dirname (str): Desired name of the folder holding all results from the different steps of recip. blastp. 
    proteome_dir (str): Path to the directory holding the indivual proteomes (and nothing else). 
    reference_proteome_path (str): Path to the proteome of the orginism from which the original forward query was derived. 
    accessions_to_incluse (list or other iterable): for querying only subset of the proteomes. Otherwise, all proteomes
                    present in './Proteomes' will be queried. 
    
    """
    
    databases  = [db for db in os.listdir(database_dir) if db != '.DS_Store']
    if not accessions_to_include:
        proteomes = [p.replace('.faa', '') for p in os.listdir(proteome_dir)]
    else:
        proteomes = accessions_to_include
    for proteome in proteomes:
        if proteome not in databases:
            raise Exception(f'{proteome} not present in databases')
    for database in databases:
        if database not in proteomes:
            raise Exception(f'{database} not present in proteomes')
            
    os.makedirs(out_dirname)
    forward_results_dir = os.path.join(out_dirname, 'forward_blast_results')
    os.makedirs(forward_results_dir)
    print(forward_results_dir)
    multi_database_query(database_dir, query_file, forward_results_dir)

    rblast_queries_dir = os.path.join(out_dirname, 'forward_best_hits')
    mq.write_best_hit_queries(proteome_dir, forward_results_dir, rblast_queries_dir)
    
    
    ref_db_name = reference_proteome_path.replace('.faa', '_blast_db')
    #os.makedirs('reference_db')
    reference_db_path = os.path.join(out_dirname, 'reference_db/reference_db')
    print(reference_db_path)
    !makeblastdb -in $reference_proteome_path -dbtype 'prot' -out $reference_db_path
    recip_blast(rblast_queries_dir, reference_db_path, os.path.join(out_dirname,'reciprocal_blast_results'))
    
  

## Run reciprocal blast

#### Run # 1 (astA2)

In [20]:
#### FILL OUT NEXT TWO LINES ################################
query_file_astA2 = './AKQ27614_1_GCA_001077675_1.faa' ##AstA2
out_dirname = './rblast_results_' + query_file_astA2.replace('.faa', '').replace('./','')

reciprocal_blastp(database_dir, query_file=query_file_astA2, out_dirname=out_dirname,
                  proteome_dir=proteome_dir, reference_proteome_path=reference_proteome_path,
                  accessions_to_include = deduped_accessions_w_palmer_strains)

#### Run # 2 (astC2)

In [19]:
#### FILL OUT NEXT TWO LINES #################################
query_file_astC2 = './AKQ27615_1_GCA_001077675_1.faa' ##AstC2
out_dirname = './rblast_results_' + query_file_astC2.replace('.faa', '').replace('./','')

reciprocal_blastp(database_dir, query_file=query_file_astC2, out_dirname=out_dirname,
                  proteome_dir=proteome_dir, reference_proteome_path=reference_proteome_path,
                  accessions_to_include = deduped_accessions_w_palmer_strains)

./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_016865465.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_024139095.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_019903195.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_014159355.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_025246195.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_001704675.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_009428985.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_013376855.2_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_022342015.1_blastpOut.txt
./rblast_results_AKQ27615_1_GCA_001077675_1/forward_blast_results/GCF_

### Processing rblast data

In [30]:
def get_blastbesthit_dict(path_to_blast_results, accession_length=13):
    """Return a dict {accession:protein_id} with the accession of genome and the best hit in the source genome"""
    
    files =os.listdir(path_to_blast_results)
    paths = [os.path.join(path_to_blast_results,file) for file in files]
    best_hits_dict = {}
    for file, path in zip(files, paths):
        accession = file[:accession_length]
        with open(path, 'r') as f:
            for i,line in enumerate(f):
                if line.startswith('>'):
                    prot_id = line.split(' ')[0][1:]
                    best_hits_dict[accession]=prot_id
                    break
    return best_hits_dict

def get_full_best_hits_dict(recip_blast_dir, accession_length=15):
    for_dir = os.path.join(recip_blast_dir, 'forward_blast_results')
    rev_dir = os.path.join(recip_blast_dir, 'reciprocal_blast_results')
    forward_bbhits = get_blastbesthit_dict(for_dir, accession_length=accession_length)
    rev_bbhits = get_blastbesthit_dict(rev_dir, accession_length=accession_length)
    full_df = pd.DataFrame.from_dict(rev_bbhits, orient='index').merge(pd.DataFrame.from_dict(forward_bbhits, orient='index'), left_index=True, right_index=True)
    full_df.columns = ['r_bbhit', 'f_bbhit']
    return full_df

astA_df = get_full_best_hits_dict('./rblast_results_AKQ27614_1', accession_length=15)
astC_df = get_full_best_hits_dict('./rblast_results_AKQ27615_1_GCA_001077675_1', accession_length=15)

In [39]:
astC_df['r_bbhit'].value_counts()

WP_005135515.1    377
WP_009518196.1     86
WP_000129946.1      4
Name: r_bbhit, dtype: int64

In [33]:
astA_df['r_bbhit'].value_counts()

WP_000972684.1    376
WP_000973732.1     89
WP_000205079.1      2
Name: r_bbhit, dtype: int64

In [40]:
astC_df['r_bbhit'].value_counts()['WP_005135515.1'] / astC_df.shape[0]

0.8072805139186295

In [41]:
astA_df['r_bbhit'].value_counts()['WP_000972684.1'] / astA_df.shape[0]

0.8051391862955032

### Save to file

In [43]:
astA_df.to_csv('./rblast_results_AKQ27614_1/results_recip_blast_astA2.csv')
astC_df.to_csv('./rblast_results_AKQ27615_1_GCA_001077675_1/results_recip_blast_astC2.csv')
